<a href="https://colab.research.google.com/github/dasha-shchep/fssh_colab/blob/master/fssh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing the FSSH algorithm on Tully's example problems
This notebook will go through the basic FSSH algorithm and run parallel trajectories as described in the original paper - Tully, J.C. *J. Chem. Phys.* (1990) **93** 1061.

In [0]:
import numpy as np
# !pip install pyscf
# from pyscf import scf

The three types of model potential that Tully described are:

A. Simple avoided crossing (SAX)

B. Dual avoided crossing (DAX)

C. Extended coupling with reflection (ECR)

On the next line we create a SAX potential:

In [0]:
# Setting up model (1-D potential) classes
# Creating a model class lets us play with more complex model potentials later
class SAXModel:
  """
  First model two-level problem defined by the following interactions in the
  diabatic representstion:
  V11(X) = A[1 - exp(-BX)] for x > 0
  V11(X) = - A[1 - exp(BX)] for x < 0
  V22(X) = - V11(X)
  V21(X) = V12(X) = C exp(-DX**2)
  Default choice of parameters:
  A = 0.01 ; B = 1.6 ; C = 0.005 ; D = 1.0
  """
  _num_states = 2
  _num_dims   = 1
  
  def __init__(self,a=0.01,b=1.6,c=0.005,d=1.0):
    self.A = a
    self.B = b
    self.C = c
    self.D = d

  def V(self,R):
    if R > 0:
      V11 = self.A*(1 - np.exp(-self.B*R))
    elif R < 0:
      V11 = - self.A*(1 - np.exp(self.B*R))
    else:
      V11=0
    V22 = -V11
    V12 = self.C*np.exp(-self.D*R**2)
    V21 = V12
    return np.array([[V11,V12],[V21,V22]])

  def dV(self,R):
    dV11 = - self.A*self.B*np.exp(-self.B*abs(R))
    dV22 = -dV11
    dV12 = self.C*np.exp(-self.D*R**2)
    dV21 = dV12
    return np.array([[dV11,dV12],[dV21,dV22]])



In [16]:
# class DAXModel:
#  def __init__(self):
#    self.representation = representation # Adiabatic or Diabatic

model = SAXModel()
model._num_states

2

In [0]:
# Generate initial conditions
numTraj = 100
initP = 0
initQ = 1

In [0]:
# Classical nuclei velocity verlet integrator

In [0]:
# Calculate hopping probability & random number generator

Trying to run CUDA on Colab GPUs

In [0]:
import torch

In [0]:
torch.cuda.memory_allocated()

0

In [0]:
a = torch.rand(128,600).cuda()

In [0]:
b = np.random.rand(128,600)

In [0]:
def func(obj):
  out = obj**2
  return out

In [0]:
%timeit func(a)

The slowest run took 1600.61 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 10.1 µs per loop


In [0]:
%timeit func(b)

The slowest run took 13.36 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 86.1 µs per loop
